In [1]:
import hashlib
import random
import string
import json
import binascii
import numpy as np
import pandas as pd
import pylab as pl
import logging
import datetime
import collections

In [2]:
import hashlib

In [3]:
! pip3 install cryptography

In [4]:
import base64
import logging

from cryptography.exceptions import UnsupportedAlgorithm
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

In [5]:
! pip3 install pyDES

In [6]:
import pyDes

# Symmetric Encryption 

use of the DES Algorithm

https://en.wikipedia.org/wiki/Data_Encryption_Standard

In [7]:
def encript(msg, key):
    k = pyDes.des(key, pyDes.CBC, "\0\0\0\0\0\0\0\0", pad=None, padmode=pyDes.PAD_PKCS5)
    return k.encrypt(msg)

In [8]:
def decript(msg, key):
    k = pyDes.des(key, pyDes.CBC, "\0\0\0\0\0\0\0\0", pad=None, padmode=pyDes.PAD_PKCS5)
    return k.decrypt(msg)

In [9]:
# Key must be 8 Bytes
key = "12345678"
msg = "Super secret message!"
encripted_msg = encript(msg, key)
print(encripted_msg)

b"\xda\xd2\xb3\xba\xe5V\xd4\x00\x89\x1fO\xd2\xe5\x13i\xf0\xf1\x98*\xec{J'\x1d"


In [10]:
decripted_msg = decript(encripted_msg, key)
print(decripted_msg)

b'Super secret message!'


# Asymmetric Cryptography

use of the RSA Algorithm (public-key cryptography)

https://en.wikipedia.org/wiki/Public-key_cryptography 

In [11]:
def generate_keys():
    # Generate new key pair
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=4096,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

In [12]:
private_key, public_key = generate_keys()

In [13]:
pem = public_key.public_bytes(encoding=serialization.Encoding.PEM, format=serialization.PublicFormat.SubjectPublicKeyInfo )
pem

b'-----BEGIN PUBLIC KEY-----\nMIICIjANBgkqhkiG9w0BAQEFAAOCAg8AMIICCgKCAgEAsKx41LCyV7jkyzMR5h8S\nDBJXZvAJxsEwc6ch4ii8zNjW459sUaL0i2Ng8DbIZLsoqZNx6nhL76ACAzPHKABD\n1fanxasKLr2aqEpUtfiHpk/LPy5+NqSRykjcDI8hBIQVUzkJS1Z+aB97fu4oFeBZ\ndqK9IPX/ip0apiac7Em28yMvkyuIpLlePrNuVrbUnHa11C7UrOz78ca+fSTseahJ\n6D5dTW7LY5y62TyM4+BlPbpiz+cHCHnh7/B5f9i8T+ouYscxj+V66ehfjmy+T4qN\nFfX/ef+6FGOIEMXvDry+WcOj+XBwuJohiRCpv+Bq8PNhxu/VckfJIIBFq4yKBGAE\nJhT9Cz9DKlsETdPDft9cR+l87MHoODHHHVuG0ZL/vVqzPsbr3IuIUDNQ8HhIlf6T\n4/2U+tcq9TPQcdZv/8TSXIHhLCnUMfLZdhPFbBcoeJH8HUGH8wVe4CFw30WSk/Ij\no5k/pT1Euup/tdNMKDD5erXRJwJ/xeTqdPlZKt+yq+9z44HYfV5i3GB7aRAS8WiQ\nReNBDgOYBWy1mLYdCPWFFfMtBmZ5M01Q6lrd0uHzrLgwJuv3giYDwdd+I5Th9Hre\nxX5f3/+YpZDEF0Q5aZOpe1VkscmVmgWy/2ZzBbBovbqoBpUwOzgQZDrt8/P96dCS\nuNWRYLS85ufH4ukxYVkQEDUCAwEAAQ==\n-----END PUBLIC KEY-----\n'

In [14]:
def encrypt_asym(plain_text , publickey):
    # Encription with the use of the RSA Library
    cipher_text_bytes = public_key.encrypt(
        plaintext=plain_text.encode('utf-8'),
        padding=padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA512(),
            label=None
        )
    )
    # Conversion of raw bytes to BASE64 representation
    cipher_text = base64.urlsafe_b64encode(cipher_text_bytes)
    return cipher_text

def decrypt_asym(cipher_text, private_key):
    # Decription with the use of the RSA Library
    decrypted_cipher_text_bytes = private_key.decrypt(
        ciphertext=base64.urlsafe_b64decode(cipher_text),
        padding=padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA512(),
            label=None
        )
    )
    # Conversion to UTF-8 Text
    decrypted_cipher_text = decrypted_cipher_text_bytes.decode('utf-8')
    return decrypted_cipher_text

In [15]:
msg = "This message was encripted with asymmetric cryptography"
encrypted_msg = encrypt_asym(msg , public_key)
encrypted_msg

b'fvjcyZR98h46NiJsSBLeM95hNKpyvwOZGj3WDc4fzKOZ6iCZ6qa-g-NstSfnT-uTGS1a7P4by6-NUuYNULXd4B5_ZNlvumgA0ziKyXT7d-hhmQCd-PLfC6Laed_qYmYS_n1mKBV9Xfhtm_CNRnknYmdeA58X3EuwtjnQaZPtj0wgK16DDCXhKKHGDtejbGxjINBalhxi18WZXM05zQsM9-OhJljZzwS_Q7bYTN2B0wIgElHKt2FGX5dRyevCIJr7CW1ack1FUaJhXFpRNIvQBsCjTiqM7FMtpiW2Md-ajxqZ3-5EyVqp6Gu3tBepd7gumcvH1UxIdJdWtABtRR2K5aD74ylB7Sit5bYf8_Vs0DT6_0ViDeAaYTATMSe1OqONre0suxo27YDiFdS_znh7cjDr4P-quuFwPNwy287lWt7cqFRiadarEILhJCnOw6X95XEr99ZKQEDuYNJMREVoMcb0Hh9OMKbpVJ308edjKqnExUCm0GxLS6zLK5jSaV_V5R7H4s5Dd0f0MjmnmdwcJtEib_As5YPrQXBKQSsXNNjdzFIV1MRUnVGsic5whVWDr1fybTrocnOeLOwT6WzwBjmZY6BPbMoZqiTwry2hG-ljnr3Tk1XMI139B9IEKEeAvXzAN191NygV-2m_AzRcuOiWfeNy_QGwOZq0FeDQVxs='

In [16]:
decripted_msg = decrypt_asym(encrypted_msg, private_key)
decripted_msg

'This message was encripted with asymmetric cryptography'

# Simple Hash Functions

https://en.wikipedia.org/wiki/Hash_function

In [17]:
msg = "This is a long contract"

In [18]:
def super_simple_hash(msg):
    """Just the first letter of the message"""
    return msg[0]

In [19]:
super_simple_hash(msg)

'T'

In [20]:
msg = "This is a totaly different contract :-)"

In [21]:
super_simple_hash(msg)

'T'

# Secure Hash Functions

https://en.wikipedia.org/wiki/Secure_Hash_Algorithms

In [22]:
def hash_msg(msg=""):
    return hashlib.sha256(str(msg).encode('utf-8')).hexdigest()

In [23]:
hash_msg("This is a long contract")

'853dbfcc24490c5b10f557aa3d35ed5ba895f2165b7cbbab561c19012665838a'

In [24]:
hash_msg("This is a totaly different contract :-)")

'fcaa045d6064a9f33c7a8677664f53ff60b0ce575979219d414a60e3129afe09'

In [25]:
def check_msg(msg, signed_hash):
    calculated_hash = hash_msg(msg)
    if signed_hash == calculated_hash:
        return True
    else:
        return False

In [26]:
msg = "This is my Contract"
msg_hash = hash_msg(msg)
check_msg(msg, msg_hash)

True

In [27]:
msg = "I change the contract so it is much better now for me :-)"
check_msg(msg, msg_hash)

False

# Digital signing messages

https://en.wikipedia.org/wiki/Digital_signature

In [28]:
def sign_msg(message, private_key):
    signature = private_key.sign(message, padding.PSS(mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH), hashes.SHA256())
    return signature

In [29]:
msg = b"A message I want to sign"
signature = sign_msg(msg, private_key)

In [30]:
signature

b'\\(\xb4\xd85O\xf3\x88\xf8\xf9o\x9dL\xe5\xa7@\xf7*\xa3\x87\xee\xb0J=\x00\x08\xfb\xd0)\xab\xec\xae+x\xb2\xa5\xe0 #O\xf1\x7fJ\xa2\x16\x0c\x81\x05z\xd6\x10\xb3\xbft\xa6\xac\x18\xe0\x03\xb9_L\'=\x10\x87\xd4H\x1a\xc7\xed\x8b\xa3\x1e\xbcn\xa1\x15sh\t\xb7\xe8\xc9\xd2\x96L\x14\xc8[\xee\xd0j1?G5\xea%1\xdeT\xd9\x12x\x0fxqt\xc4\xd6\xaa\x8a\xc5rd\xc9\xc6\xa2\x8a\xddJ\x04e\x0f\x1a-\x84\x90\x19\x91\xb5\xb6\x92E\xc97\x19\x15x\xa1\xbcU\xd1\xed#\xb2(#\xe6\xd6\x073EE\x89\x08L\x99s*\xa6x\xa3\x99\xe1;h\xe7\xcaI%8_V\x97\x93\x99=7\xfa\xa6\x91@\xeb;\xd05h\xf2\x0e$\xed\x0e\xe4\x06\'\xa0\xaf\x0f\x9e\xb5\x9b\xeb\xbe\xd4\x84K\x81\xb4V\xb7\x10\x1a\x11\x8d4\x8a{\xce\x9a"P\xc1\xd2\x8e1&\x1b+)\xf1\x10\x92[;\xec\x8e]>\rw}\xaa\xf4v\xec\x9bT_\xa6\xc3\xcfbv\xa2\x1f)\xb8\xabc\xe8\xb1\xfe\xa6\x12Y*\x1d\xc9\xe5\xe4\xd4\x02\xd88\x03\xb5_W\x9a\xce\xfcQD\x90\xa3\x8d&{\xc0tr\xc0\xb1\xb3\xe1L\xb9\xd1\x8c\xe3\x82|\x96\xa6U\xbd]tIX\xbb\xda\xf93\xf0FO\xe9B^*]D\x8f\x96\x87\x9c0\x14\xe8\xb4\xa7\xd3z\xd1\xec\r\xdb\xec\x89\x86XZ\xc1\

In [31]:
def verify(signature, message, public_key):
    try:
        public_key.verify(signature, message, padding.PSS( mgf=padding.MGF1(hashes.SHA256()), salt_length=padding.PSS.MAX_LENGTH), hashes.SHA256())
        return True
    except: 
        return False

In [32]:
verify(signature, msg, public_key)

True

In [33]:
another_msg = "Another Message"

In [ ]:
verify(signature, another_msg, public_key)

False

In [ ]:
bob_private_key, bob_public_key = generate_keys()

In [ ]:
verify(signature, msg, bob_public_key)